In [281]:
!pip install numpy pandas matplotlib seaborn scikit-learn xgboost




In [282]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import math

In [283]:
bm_data = pd.read_csv("Train.csv")

In [284]:
bm_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [285]:
bm_data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [286]:
bm_data['Item_Weight'].fillna(bm_data['Item_Weight'].mean(),inplace=True)

C:\Users\91954\AppData\Local\Temp\ipykernel_15264\3305432237.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bm_data['Item_Weight'].fillna(bm_data['Item_Weight'].mean(),inplace=True)


In [287]:
mode_outlet = bm_data.pivot_table(values='Outlet_Size',columns = 'Outlet_Type',aggfunc = (lambda x:x.mode([0])))

In [288]:
miss_values = bm_data['Outlet_Size'].isnull()

In [289]:
miss_values

0       False
1       False
2       False
3        True
4       False
        ...  
8518    False
8519     True
8520    False
8521    False
8522    False
Name: Outlet_Size, Length: 8523, dtype: bool

In [290]:
bm_data.loc[miss_values,'Outlet_Size']=bm_data.loc[miss_values,'Outlet_Type'].apply(lambda x:mode_outlet[x])

In [291]:
bm_data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [292]:
#PREPROCESSING
rem_cols=[
    'Item_Identifier',
    'Item_Type',
    'Outlet_Identifier',
    'Outlet_Establishment_Year'
]
bm_data = bm_data.drop(rem_cols,axis=1)

In [293]:
bm_data.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,249.8092,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,48.2692,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,141.6180,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,182.0950,Small,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,53.8614,High,Tier 3,Supermarket Type1,994.7052


In [294]:
bm_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [295]:
bm_data.replace({'Item_Fat_Content':{'LF':'Low Fat','low fat':'Low Fat','reg':"Regular"}},inplace=True)

In [296]:
bm_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [297]:
bm_data['Outlet_Size'].value_counts()

Outlet_Size
Small     4798
Medium    2793
High       932
Name: count, dtype: int64

In [298]:
bm_data['Outlet_Location_Type'].value_counts()

Outlet_Location_Type
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: count, dtype: int64

In [299]:
bm_data['Outlet_Type'].value_counts()

Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: count, dtype: int64

In [300]:
#Label Encoding
enc = LabelEncoder()

In [301]:
bm_data['Item_Fat_Content'] = enc.fit_transform(bm_data['Item_Fat_Content'])
bm_data['Outlet_Size'] = enc.fit_transform(bm_data['Outlet_Size'])
bm_data['Outlet_Location_Type'] = enc.fit_transform(bm_data['Outlet_Location_Type'])
bm_data['Outlet_Type'] = enc.fit_transform(bm_data['Outlet_Type'])


In [302]:
bm_data.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,0,0.016047,249.8092,1,0,1,3735.1380
1,5.92,1,0.019278,48.2692,1,2,2,443.4228
2,17.50,0,0.016760,141.6180,1,0,1,2097.2700
3,19.20,1,0.000000,182.0950,2,2,0,732.3800
4,8.93,0,0.000000,53.8614,0,2,1,994.7052


In [303]:
bm_data['Outlet_Type'].value_counts()

Outlet_Type
1    5577
0    1083
3     935
2     928
Name: count, dtype: int64

In [304]:
bm_data['Outlet_Location_Type'].value_counts()

Outlet_Location_Type
2    3350
1    2785
0    2388
Name: count, dtype: int64

In [305]:
bm_data['Outlet_Size'].value_counts()

Outlet_Size
2    4798
1    2793
0     932
Name: count, dtype: int64

In [306]:
bm_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
0    5517
1    3006
Name: count, dtype: int64

In [307]:
bm_data=bm_data.drop(columns='Item_Fat_Content',axis=1)

In [308]:
X = bm_data.drop(columns='Item_Outlet_Sales',axis=1)
Y = bm_data['Item_Outlet_Sales']

In [309]:
X

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.300,0.016047,249.8092,1,0,1
1,5.920,0.019278,48.2692,1,2,2
2,17.500,0.016760,141.6180,1,0,1
3,19.200,0.000000,182.0950,2,2,0
4,8.930,0.000000,53.8614,0,2,1
...,...,...,...,...,...,...
8518,6.865,0.056783,214.5218,0,2,1
8519,8.380,0.046982,108.1570,2,1,1
8520,10.600,0.035186,85.1224,2,1,1
8521,7.210,0.145221,103.1332,1,2,2


In [310]:
Y

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [311]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2)

In [312]:
X_train

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type
7173,11.800000,0.057422,149.9366,2,0,1
3315,12.857645,0.000000,100.1384,1,2,3
5932,7.725000,0.047783,249.1092,2,0,1
7872,10.500000,0.052555,89.6830,2,0,1
5946,12.857645,0.235859,46.1402,2,0,0
...,...,...,...,...,...,...
1099,7.235000,0.099963,193.8452,2,0,1
2514,19.350000,0.118103,224.8088,2,0,1
6637,18.700000,0.092096,183.3292,2,1,1
2575,8.880000,0.086957,152.5682,2,1,1


In [313]:
X_test

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type
1112,12.857645,0.099747,75.2328,1,2,3
1751,12.150000,0.131446,246.8460,2,1,1
7648,5.800000,0.131221,87.8172,2,1,1
7362,17.000000,0.087388,125.0730,2,2,0
5332,12.857645,0.009585,102.5016,2,0,0
...,...,...,...,...,...,...
3503,12.857645,0.037505,126.2704,1,2,3
975,20.000000,0.028118,46.7744,2,1,1
6190,19.100000,0.178273,172.4422,2,1,1
32,18.700000,0.000000,256.6672,1,2,2


In [314]:
Y_train

7173    1662.5026
3315    2956.1520
5932    2490.0920
7872     988.7130
5946      45.9402
          ...    
1099    1957.4520
2514    2013.3792
6637    2006.7212
2575    1372.2138
7336    1830.9500
Name: Item_Outlet_Sales, Length: 6818, dtype: float64

In [315]:
Y_test

1112    1544.6560
1751    6404.9960
7648    1070.6064
7362     369.5190
5332     101.2016
          ...    
3503    4255.7936
975     1222.4088
6190    1551.9798
32      3068.0064
4433    5480.8656
Name: Item_Outlet_Sales, Length: 1705, dtype: float64

In [316]:
#ML models
# 1. XGBoost Regressor
xg = XGBRegressor(n_estimators=1000)

In [317]:
xg.fit(X_train,Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [318]:
pred = xg.predict(X_train)

In [319]:
r2_tr = metrics.r2_score(Y_train,pred)
print("Training R2 score for XG : ",r2_tr)

Training R2 score for XG :  0.9907960982304379


In [338]:
xg_pred = xg.predict(X_test)
r2_tr_rf2 = metrics.r2_score(Y_test,xg_pred)
print("Testing R2 score for XG : ",r2_tr_rf2)


Testing R2 score for XG :  0.4008859855823267


In [321]:
from sklearn.ensemble import RandomForestRegressor

In [322]:
rfr = RandomForestRegressor(n_estimators=1000)

In [323]:
rfr.fit(X_train,Y_train)

RandomForestRegressor(n_estimators=1000)

In [324]:
rf_pred = rfr.predict(X_train)
r2_tr_rf = metrics.r2_score(Y_train,rf_pred)
print("Training R2 score for RF : ",r2_tr_rf)

Training R2 score for RF :  0.9387562399553475


In [325]:
rf_pred1 = rfr.predict(X_test)
r2_tr_rf1 = metrics.r2_score(Y_test,rf_pred1)
print("Testing R2 score for RF : ",r2_tr_rf1)

Testing R2 score for RF :  0.5426254472055192


In [326]:
from xgboost import XGBRFRegressor

In [327]:
xgrf = XGBRFRegressor(n_estimators=1000)
xgrf.fit(X_train,Y_train)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=1000, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg_alpha=None, ...)

In [328]:
xgrf_pred = xgrf.predict(X_train)
r2_tr_xgrf = metrics.r2_score(Y_train,xgrf_pred)
print("Training R2 score for XGRF : ",r2_tr_xgrf)

Training R2 score for XGRF :  0.6237959282850528


In [329]:
xgrf_pred1 = xgrf.predict(X_test)
r2_tr_xgrf1 = metrics.r2_score(Y_test,xgrf_pred1)
print("Testing R2 score for XGRF : ",r2_tr_xgrf1)

Testing R2 score for XGRF :  0.5826972702297802


In [330]:
print("XGBoost Regressor metrics")
xg_mae = mean_absolute_error(xg_pred,Y_test) 
xg_mse = mean_squared_error(xg_pred,Y_test)
xg_rmse = math.sqrt(xg_mse)
print("MAE : ", xg_mae)
print("MSE : ", xg_mse)
print("RMSE : ", xg_rmse)
print("R2_Score : ",r2_tr_rf2)


XGBoost Regressor metrics
MAE :  963.9370940260239
MSE :  1849509.7225939557
RMSE :  1359.966809372183
R2_Score :  0.4008859855823267


In [331]:
print("Random Forest Regressor metrics")
rf_mae = mean_absolute_error(rf_pred1,Y_test) 
rf_mse = mean_squared_error(rf_pred1,Y_test)
rf_rmse = math.sqrt(rf_mse)
print("MAE : ", rf_mae)
print("MSE : ", rf_mse)
print("RMSE : ", rf_rmse)
print("R2_Score : ", r2_tr_rf1)

Random Forest Regressor metrics
MAE :  831.8013987989441
MSE :  1411949.4151420752
RMSE :  1188.2547770331391
R2_Score :  0.5426254472055192


In [332]:
print("XGBRF Regressor metrics")
xgbrf_mae = mean_absolute_error(xgrf_pred1,Y_test) 
xgbrf_mse = mean_squared_error(xgrf_pred1,Y_test)
xgbrf_rmse = math.sqrt(xgbrf_mse)
print("MAE : ", xgbrf_mae)
print("MSE : ", xgbrf_mse)
print("RMSE : ", xgbrf_rmse)
print("R2_Score : ",r2_tr_xgrf1 )

XGBRF Regressor metrics
MAE :  791.3948872383829
MSE :  1288244.7036816506
RMSE :  1135.0086800027789
R2_Score :  0.5826972702297802


In [333]:
#Normalisation process

max_mae = max(xg_mae,rf_mae,xgbrf_mae)
max_mse = max(xg_mse,rf_mse,xgbrf_mse)
max_rmse = max(xg_rmse,rf_rmse,xgbrf_rmse)
print(max_mae,' ',max_mse,' ',max_rmse)

963.9370940260239   1849509.7225939557   1359.966809372183


In [334]:
print('After Normalisation process the Metric_Values of each model are ')
print()
print('Xgboost metrics : ')
print('MAE : ',(xg_mae/max_mae),'\nMSE : ',(xg_mse/max_mse),'\nRMSE : ',(xg_rmse/max_rmse),'\nR2 : ',r2_tr_rf2)

print('\nRFR metrics : ')
print('MAE : ',(rf_mae/max_mae),'\nMSE : ',(rf_mse/max_mse),'\nRMSE : ',(rf_rmse/max_rmse),'\nR2 : ',r2_tr_rf1)

print('\nXgbRF metrics : ')
print('MAE : ',(xgbrf_mae/max_mae),'\nMSE : ',(xgbrf_mse/max_mse),'\nRMSE : ',(xgbrf_rmse/max_rmse),'\nR2 : ',r2_tr_xgrf1)

After Normalisation process the Metric_Values of each model are 

Xgboost metrics : 
MAE :  1.0 
MSE :  1.0 
RMSE :  1.0 
R2 :  0.4008859855823267

RFR metrics : 
MAE :  0.8629208316123661 
MSE :  0.7634182172136961 
RMSE :  0.8737380712854946 
R2 :  0.5426254472055192

XgbRF metrics : 
MAE :  0.8210026278094629 
MSE :  0.6965330800612795 
RMSE :  0.8345855738396629 
R2 :  0.5826972702297802


In [335]:
import pickle

In [336]:
pickle.dump(xgrf,open('sales_prd.pkl','wb'))